# graphs and sessions

following along with [this page](https://www.tensorflow.org/programmers_guide/graphs)

the general idea is that a `tensorflow` computation is defined as a dag (the `tf.Graph`) and executed within a computation environment called a session (the `tf.Session`)

I'm not yet sure how this will differ content-wise from the number of other introductions we've already encountered

In [1]:
import tensorflow as tf

import utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## why dataflow graphs?

"dataflow graphs" are the names this document uses to describe the dag that everywhere else is referred to as a graph or computation graph. they provide the following example image

![image.png](https://www.tensorflow.org/images/tensors_flowing.gif)

the author describes [*dataflow*](https://en.wikipedia.org/wiki/Dataflow_programming) as "a common programming model for parallel computing." here they describe the nodes as "units of computation" and the edges as the "data consumed or produced by a computation."

the example: the `tf.matmul` is an operation (a node) which has two inpcoming edges (the matrices being multiplied) and one outgoing edge (the output of the matrix multiplication calculation).

the listed advantages:

1. parallelism
1. distributed execution
1. compilation
1. portability

## what is a `tf.Graph`?

the graph is basically two things:

1. graph structure: nodes and edges
1. graph collections: metadata information grouped into categories / collections

this section is *not* helpful. the distinction between the above is not at all clear, and they really only toss out there that "nodes and edges" are not the computation. the analogy is "assembly code", suggesting it "does not contain all of the userful context the source code conveys"

## building a `tf.Graph`

first stage of a tensorflow project: building the `tf.Graph`. tensorflow itself provides a *default graph* that all api functions take as an implicit input (okay, this is good to know). discussion on having more than one graph is at the end of this page.

the example are pretty good and worth reading so I'll reproduce them exactly here. I like the way they break it down into the `tf.Operation` and `tf.Tensor` objects that are being created under the hood:

> + Calling `tf.constant(42.0)` creates a single `tf.Operation` that produces the value 42.0, adds it to the default graph, and returns a `tf.Tensor` that represents the value of the constant.
> + Calling `tf.matmul(x, y)` creates a single `tf.Operation` that multiplies the values of `tf.Tensor` objects `x` and `y`, adds it to the default graph, and returns a `tf.Tensor` that represents the result of the multiplication.
> + Executing `v = tf.Variable(0)` adds to the graph a `tf.Operation` that will store a writeable tensor value that persists between `tf.Session.run` calls. The `tf.Variable` object wraps this operation, and can be used like a tensor, which will read the current value of the stored value. The `tf.Variable` object also has methods such as `assign` and `assign_add` that create `tf.Operation` objects that, when executed, update the stored value. (See Variables for more information about variables.)
> + Calling `tf.train.Optimizer.minimize` will add operations and tensors to the default graph that calculate gradients, and return a `tf.Operation` that, when run, will apply those gradients to a set of variables.

In [2]:
x = tf.constant(10)
y = tf.Variable(0)
z = tf.multiply(x, y)

In [6]:
import importlib
importlib.reload(utils)

<module 'utils' from '/home/zlamberty/notebooks/deep_learning_world_tour/tensorflow_tutorial/utils.py'>

In [7]:
with tf.Session() as sess:
    y.assign(5)
    graph_def = z.graph.as_graph_def()
    tmp_def = utils.rename_nodes(graph_def, lambda s: "/".join(s.split('_', 1)))
    utils.show_graph(tmp_def)

## naming operations

all `tf.Operation` or `tf.Tensor` objects in a `tf.Graph` are put into namespaces. tensorflow automatically creates unique names in the root namespace for any operation that is not named explicitly, but explicit names cary the obvious readability / maintainability benefits.

ways to name:

+ hand-crafted names can be provided to all objects and tensors via the `name` parameter.
+ name scope prefixes can be added with `tf.name_scope` context managers

In [4]:
c_0 = tf.constant(0, name="c")  # => operation named "c"

# Already-used names will be "uniquified".
c_1 = tf.constant(2, name="c")  # => operation named "c_1"

# Name scopes add a prefix to all operations created in the same context.
with tf.name_scope("outer"):
    c_2 = tf.constant(2, name="c")  # => operation named "outer/c"
  
    # Name scopes nest like paths in a hierarchical file system.
    with tf.name_scope("inner"):
        c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"
  
    # Exiting a name scope context will return to the previous prefix.
    c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"
  
    # Already-used name scopes will be "uniquified".
    with tf.name_scope("inner"):
        c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"

In [5]:
[_.name for _ in [c_0, c_1, c_2, c_3, c_4, c_5]]

['c:0',
 'c_1:0',
 'outer/c:0',
 'outer/inner/c:0',
 'outer/c_1:0',
 'outer/inner_1/c:0']

## placing operations on different devices

we've covered this about a million times, but there is actually new info here. first, the repeat info: instruct a specific graph calculation to occur on a specific device with the `tf.device` context manager.

the defice specification has the format

```python
'/job:{job_name}/task:{task_index}/device:{device_type}:{device_index}'
```

+ `job_name` is an alpha-numeric string that doesn't start with a number
+ `task_index` is a non-negative integer indexing tasks within a fixed `job_name`
+ `device_type` is `gpu`, `cpu`, `tpu`, etc
+ `device_index` is a non-negative integer indexing devices within a type category `device_type`

here's an example program for distributing computation utilizing both a gpu and cpu (more specifically, this utilizes either gpu or cpu unless otherwise specified, but *requires* cpu or gpu usage where listed explicitly):

```python
# Operations created outside either context will run on the "best possible"
# device. For example, if you have a GPU and a CPU available, and the operation
# has a GPU implementation, TensorFlow will choose the GPU.
weights = tf.random_normal(...)

with tf.device("/device:CPU:0"):
  # Operations created in this context will be pinned to the CPU.
  img = tf.decode_jpeg(tf.read_file("img.jpg"))

with tf.device("/device:GPU:0"):
  # Operations created in this context will be pinned to the GPU.
  result = tf.matmul(weights, img)
```

and here, another example where we have a distributed setup with a parameter sever job (`/job:ps`) and a worker job (`/job:worker`)

```python
with tf.device("/job:ps/task:0"):
    weights_1 = tf.Variable(tf.truncated_normal([784, 100]))
    biases_1 = tf.Variable(tf.zeroes([100]))

with tf.device("/job:ps/task:1"):
    weights_2 = tf.Variable(tf.truncated_normal([100, 10]))
    biases_2 = tf.Variable(tf.zeroes([10]))

with tf.device("/job:worker"):
    layer_1 = tf.matmul(train_batch, weights_1) + biases_1
    layer_2 = tf.matmul(train_batch, weights_2) + biases_2
```

general heuristic the author recommends: use `tf.train.replica_device_setter` in conjunction with `tf.device` for distributed computing:

```python
with tf.device(tf.train.replica_device_setter(ps_tasks=3)):
    # tf.Variable objects are, by default, placed on tasks in "/job:ps" in a
    # round-robin fashion.
    w_0 = tf.Variable(...)  # placed on "/job:ps/task:0"
    b_0 = tf.Variable(...)  # placed on "/job:ps/task:1"
    w_1 = tf.Variable(...)  # placed on "/job:ps/task:2"
    b_1 = tf.Variable(...)  # placed on "/job:ps/task:0"
  
    input_data = tf.placeholder(tf.float32)     # placed on "/job:worker"
    layer_0 = tf.matmul(input_data, w_0) + b_0  # placed on "/job:worker"
    layer_1 = tf.matmul(layer_0, w_1) + b_1     # placed on "/job:worker"
```

## tensor-like objects

there are some things that are not tensor that can be cast to tensors. they call this "tensor-like". the objecdts they list are:

+ `tf.Tensor`
+ `tf.Variable`
+ `numpy.ndarray`
+ `list`
+ any scalar `python` builtin type

if I have something special I want to make "tensor-like" I can register a conversion method using `tf.register_tensor_conversion_function`

## executing a graph in a `tf.Session`

the `tf.Session` object manages a connection between the "client" program (`python`) and the `c++` runtime. it handles distirbution, caching, etc.

### creating a `tf.Session`

we've done this about a million times now:

```python
with tf.Session() as sess:
    ...
```

remotely, the protocol is [`grpc`](https://grpc.io/), so you can make remote connections to tensorflow sessions using:

```python
with tf.Session('grps://{hostname}:{port}'):
    ...
```

in the documentation the example port is 2222, so that is possibly a tensorflow default

there is a note added that higher-level apis will often handle sessions for you, so to parameterize them you must utilize provided `target` or `config` arguments instead of the low-level arguments to `tf.Session`

In [6]:
tf.Session.__init__?

constructor arguments:

+ `target`: the location of the execution engine. default is localhost device
+ `graph`: optional explicit graph argument. default behavior is default session graph
+ `config`: this is a `ConfigProto` object, a protobuffer configuring the running of the current session
    + some options:
        + `graph_options.optimizer_options`: hard-coded options for the optimizer operations
        + `gpu_options.allow_growth`: rather than allocate most GPU memory at startup, allow it to grow as memory is required

### using `tf.Session.run` to execute operations

the input to a `tf.Session.run` call is a list of *fetches*, tensor-like objects for which you would like the computation to be performed. this determines a *subgraph* of the overall session graph that must be calculated (it will be lazy and avoid calculating anything not on the critical path).

here is their provided example demonstrating how different arguments lead to different subgraphs being calculated

In [10]:
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
output = tf.nn.softmax(y)
init_op = w.initializer

with tf.Session() as sess:
    # Run the initializer on `w`.
    sess.run(init_op)
    
    # evaluate just x -- won't calculate y or output
    print(sess.run(x))
    
    # Evaluate `output`. `sess.run(output)` will return a NumPy array containing
    # the result of the computation.
    print(sess.run(output))
    
    # Evaluate `y` and `output`. Note that `y` will only be computed once, and its
    # result used both to return `y_val` and as an input to the `tf.nn.softmax()`
    # op. Both `y_val` and `output_val` will be NumPy arrays.
    y_val, output_val = sess.run([y, output])

[[ 37. -23.]
 [  1.   4.]]
[[1.0000000e+00 6.7093851e-11]
 [5.4350060e-01 4.5649943e-01]]


parameterization also comes to individual `tf.Session` objects through the `feed_dict` argument:

In [13]:
# Define a placeholder that expects a vector of three floating-point values,
# and a computation that depends on it.
x = tf.placeholder(tf.float32, shape=[3])
y = tf.square(x)

with tf.Session() as sess:
    # Feeding a value changes the result that is returned when you evaluate `y`.
    print(sess.run(y, {x: [1.0, 2.0, 3.0]}))  # => "[1.0, 4.0, 9.0]"
    print(sess.run(y, {x: [0.0, 0.0, 5.0]}))  # => "[0.0, 0.0, 25.0]"
  
    # Raises <a href="../api_docs/python/tf/errors/InvalidArgumentError"><code>tf.errors.InvalidArgumentError</code></a>, because you must feed a value for
    # a `tf.placeholder()` when evaluating a tensor that depends on it.
    try:
        sess.run(y)
    except tf.errors.InvalidArgumentError:
        print('told ya so!')
  
    # Raises `ValueError`, because the shape of `37.0` does not match the shape
    # of placeholder `x`.
    try:
        sess.run(y, {x: 37.0})
    except ValueError:
        print('told ya so!')

[1. 4. 9.]
[ 0.  0. 25.]
told ya so!
told ya so!


one last example, this time demonstrating how to use the `options` and `run_metadata` arguments:

In [28]:
y = tf.matmul([[37.0, -23.0], [1.0, 4.0]], tf.random_uniform([2, 2]))

with tf.Session() as sess:
    # Define options for the `sess.run()` call.
    options = tf.RunOptions()
    options.output_partition_graphs = True
    options.trace_level = tf.RunOptions.FULL_TRACE
  
    # Define a container for the returned metadata.
    metadata = tf.RunMetadata()
  
    sess.run(y, options=options, run_metadata=metadata)
  
    # Print the subgraphs that executed on each device.
    print(metadata.partition_graphs)
  
    print('\n{:-^100}\n'.format(''))
    
    # Print the timings of each operation that executed.
    print(metadata.step_stats)

[node {
  name: "MatMul_11/a"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\024B\000\000\270\301\000\000\200?\000\000\200@"
      }
    }
  }
}
node {
  name: "random_uniform_11/shape"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
          dim {
            size: 2
          }
        }
        tensor_content: "\002\000\000\000\002\000\000\000"
      }
    }
  }
}
node {
  name: "random_uniform_11/RandomUniform"
  op: "RandomUniform"
  input: "random_uniform_11/shape

In [15]:
options = tf.RunOptions()

In [17]:
[_ for _ in dir(options) if not _[:2] == '__']

['ByteSize',
 'Clear',
 'ClearExtension',
 'ClearField',
 'CopyFrom',
 'DEBUG_OPTIONS_FIELD_NUMBER',
 'DESCRIPTOR',
 'DiscardUnknownFields',
 'Extensions',
 'FULL_TRACE',
 'FindInitializationErrors',
 'FromString',
 'HARDWARE_TRACE',
 'HasExtension',
 'HasField',
 'INTER_OP_THREAD_POOL_FIELD_NUMBER',
 'IsInitialized',
 'ListFields',
 'MergeFrom',
 'MergeFromString',
 'NO_TRACE',
 'OUTPUT_PARTITION_GRAPHS_FIELD_NUMBER',
 'ParseFromString',
 'REPORT_TENSOR_ALLOCATIONS_UPON_OOM_FIELD_NUMBER',
 'RegisterExtension',
 'SOFTWARE_TRACE',
 'SerializePartialToString',
 'SerializeToString',
 'SetInParent',
 'TIMEOUT_IN_MS_FIELD_NUMBER',
 'TRACE_LEVEL_FIELD_NUMBER',
 'TraceLevel',
 'WhichOneof',
 '_CheckCalledFromGeneratedFile',
 '_SetListener',
 '_extensions_by_name',
 '_extensions_by_number',
 '_tf_api_names']

## visualizing your graph

the recommended tool for visualizing graphs is the *grpah visualizer*, a component of the tensorboard program. they recommend creating a `tf.summary.FileWriter` object and using that to write summary files to a monitored tensorboard directory in a way that tensorboard can turn into a graph.

this has been hacked out and put into a single function in `utils`

In [34]:
import matplotlib
import pandas as pd

print(matplotlib.__file__)
print(pd.__file__)

/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py
/usr/local/lib/python3.5/dist-packages/pandas/__init__.py


## programming with multiple graphs

it is common that you will want to keep the training, evaluation, and prediction stages of your model highly separated. this could be done by using separate python processes, but it can *also* be done by using completely independent graphs.

`tf.Graph` objects define independent namespaces for `tf.Operations` objects. within that namespace, all operations are "uniquified". if you have several unconnected computational graphs, consider creating separate `tf.Graph` objects and adding connected components to their own graph. this is organizationally better, but also reduces the overhead of storing everything in the default graph.

finally, replace the default graph with the `tf.Graph.as_default` context manager

In [39]:
g_1 = tf.Graph()
g_1

In [40]:
with g_1.as_default():
    # Operations created in this scope will be added to `g_1`.
    c = tf.constant("Node in g_1")
  
    # Sessions created in this scope will run operations from `g_1`.
    sess_1 = tf.Session()

In [41]:
g_2 = tf.Graph()
g_2

In [42]:
with g_2.as_default():
    # Operations created in this scope will be added to `g_2`.
    d = tf.constant("Node in g_2")

In [43]:
# Alternatively, you can pass a graph when constructing a <a href="../api_docs/python/tf/Session"><code>tf.Session</code></a>:
# `sess_2` will run operations from `g_2`.
sess_2 = tf.Session(graph=g_2)

In [44]:
assert c.graph is g_1
assert sess_1.graph is g_1

assert d.graph is g_2
assert sess_2.graph is g_2

In [54]:
g_1.get_operations()

[<tf.Operation 'Const' type=Const>]

In [48]:
g_2.get_operations()

[<tf.Operation 'Const' type=Const>]

In [56]:
g = tf.get_default_graph()
print(g.get_operations())

[<tf.Operation 'Const' type=Const>, <tf.Operation 'Variable/initial_value' type=Const>, <tf.Operation 'Variable' type=VariableV2>, <tf.Operation 'Variable/Assign' type=Assign>, <tf.Operation 'Variable/read' type=Identity>, <tf.Operation 'Mul' type=Mul>, <tf.Operation 'Assign/value' type=Const>, <tf.Operation 'Assign' type=Assign>, <tf.Operation 'c' type=Const>, <tf.Operation 'c_1' type=Const>, <tf.Operation 'outer/c' type=Const>, <tf.Operation 'outer/inner/c' type=Const>, <tf.Operation 'outer/c_1' type=Const>, <tf.Operation 'outer/inner_1/c' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'random_uniform/shape' type=Const>, <tf.Operation 'random_uniform/min' type=Const>, <tf.Operation 'random_uniform/max' type=Const>, <tf.Operation 'random_uniform/RandomUniform' type=RandomUniform>, <tf.Operation 'random_uniform/sub' type=Sub>, <tf.Operation 'random_uniform/mul' type=Mul>, <tf.Operation 'random_uniform' type=Add>, <tf.Operation 'Variable_1' type=VariableV2>, <tf.Operati

In [45]:
sess_1.close()
sess_2.close()

# summary

+ tensorflow is designed to have graphs of calculations and a session / engine in which to execute them
+ the graph interface is a class `tf.Graph`
    + one exists by default and all tensorflow items are added to it
    + you can create separate graphs using `tf.Graph` directly
+ operations and tensors have names and those names may possibly differentiate *scopes*, localized namespaces
+ tensorflow gives you control over the hardware and networked resources used for a computation
+ operations are executed in sessions
    + sessions can be configured
    + sessions can be running on remote servers
+ tensorboard allows you to visualize graphs as diagrams